In [1]:
from webdriver_manager.chrome import ChromeDriverManager
ChromeDriverManager().install()


/Users/mac/Desktop/ozcoding/Database/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


'/Users/mac/.wdm/drivers/chromedriver/mac64/129.0.6668.89/chromedriver-mac-x64/chromedriver'

In [2]:
from selenium import webdriver
browser = webdriver.Chrome()


In [3]:
url = 'https://google.com'
browser.get(url)

In [4]:
from selenium.webdriver.common.by import By
browser.find_element(By.CLASS_NAME, 'uU7dJb').text

'대한민국'

In [5]:
url = 'https://www.yes24.com/Product/category/bestseller?CategoryNumber=001&sumgb=06'
browser.get(url)

In [6]:
# 1페이지 링크 데이터 전부 수집

### 한 개의 베스트 셀러 링크 데이터 수집
browser.find_element(By.CLASS_NAME, 'gd_name').get_attribute('href')

### 1페이지 전체의 링크 데이터 
datas = browser.find_elements(By.CLASS_NAME, 'gd_name')

for i in datas :
    print(i.get_attribute('href'))

https://www.yes24.com/Product/Goods/133213071
https://www.yes24.com/Product/Goods/125557465
https://www.yes24.com/Product/Goods/133312039
https://www.yes24.com/Product/Goods/131930939
https://www.yes24.com/Product/Goods/133870359
https://www.yes24.com/Product/Goods/104663596
https://www.yes24.com/Product/Goods/133236068
https://www.yes24.com/Product/Goods/133308510
https://www.yes24.com/Product/Goods/133199014
https://www.yes24.com/Product/Goods/132560712
https://www.yes24.com/Product/Goods/134331993
https://www.yes24.com/Product/Goods/117014613
https://www.yes24.com/Product/Goods/128266166
https://www.yes24.com/Product/Goods/124539805
https://www.yes24.com/Product/Goods/8759796
https://www.yes24.com/Product/Goods/124999476
https://www.yes24.com/Product/Goods/133819718
https://www.yes24.com/Product/Goods/129094064
https://www.yes24.com/Product/Goods/132565140
https://www.yes24.com/Product/Goods/129602366
https://www.yes24.com/Product/Goods/133813184
https://www.yes24.com/Product/Goods/

In [7]:
# 3페이지 까지의 링크 데이터 전부 수집
import time

link_list = []
for i in range (1, 4) :
    print('*' * 10, f"현재 {i} 페이지 수집 중입니다.", "*" * 10)
    url = f'https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber={i}&pageSize=24'
    browser.get(url)

    browser.find_element(By.CLASS_NAME, 'gd_name').get_attribute('href')

    datas = browser.find_elements(By.CLASS_NAME, 'gd_name')

    for i in datas : 
        link = i.get_attribute('href')
        link_list.append(link)

    time.sleep(3)

print(link_list)


********** 현재 1 페이지 수집 중입니다. **********
********** 현재 2 페이지 수집 중입니다. **********
********** 현재 3 페이지 수집 중입니다. **********
['https://www.yes24.com/Product/Goods/133213071', 'https://www.yes24.com/Product/Goods/125557465', 'https://www.yes24.com/Product/Goods/133312039', 'https://www.yes24.com/Product/Goods/131930939', 'https://www.yes24.com/Product/Goods/133870359', 'https://www.yes24.com/Product/Goods/104663596', 'https://www.yes24.com/Product/Goods/133236068', 'https://www.yes24.com/Product/Goods/133308510', 'https://www.yes24.com/Product/Goods/133199014', 'https://www.yes24.com/Product/Goods/132560712', 'https://www.yes24.com/Product/Goods/134331993', 'https://www.yes24.com/Product/Goods/117014613', 'https://www.yes24.com/Product/Goods/128266166', 'https://www.yes24.com/Product/Goods/124539805', 'https://www.yes24.com/Product/Goods/8759796', 'https://www.yes24.com/Product/Goods/124999476', 'https://www.yes24.com/Product/Goods/133819718', 'https://www.yes24.com/Product/Goods/129094064',

In [8]:
len(link_list)

72

In [9]:
#for link in link_list :
    #상세 페이지로 이동


browser.get(link_list[0])

title = browser.find_element(By.CLASS_NAME, 'gd_name').text
author = browser.find_element(By.CLASS_NAME, 'gd_auth').text
publisher = browser.find_element(By.CLASS_NAME, 'gd_pub').text
publishing = browser.find_element(By.CLASS_NAME, 'gd_date').text
rating = browser.find_element(By.CLASS_NAME, "yes_b").text
review = browser.find_element(By.CLASS_NAME, "gd_reviewCount").text
sales = browser.find_element(By.CLASS_NAME, 'gd_sellNum').text.split(" ")[2]
price = browser.find_element(By.CLASS_NAME, "yes_m").text[:-1]
ranking = browser.find_element(By.CLASS_NAME, "gd_best").text.split(" | ")[0]
ranking_weeks = browser.find_element(By.CLASS_NAME, "gd_best").text.split(" | ")[1]


In [10]:
# 데이터 베이스 연동 후 -> 수집한 데이터를 db에 저장 (csv)
import pymysql
import time
import re
from datetime import datetime


conn = pymysql.connect(
    host='localhost',
    user='root',
    password='dudgus724',
    db='yes24',
    charset='utf8mb4',
    cursorclass=pymysql.cursors.DictCursor

)

with conn.cursor() as cur :
    for link in link_list:

        browser.get(link)

        title = browser.find_element(By.CLASS_NAME, 'gd_name').text
        author = browser.find_element(By.CLASS_NAME, 'gd_auth').text
        publisher = browser.find_element(By.CLASS_NAME, 'gd_pub').text

        # 날짜 형식 바꾸기
        publishing = browser.find_element(By.CLASS_NAME, 'gd_date').text
        match = re.search(r'(\d+)년 (\d+)월 (\d+)일', publishing)

        if match:
            year, month, day = match.groups()
            data_obj = datetime(int(year), int(month), int(day))
            publishing = data_obj.strftime("%Y-%m-%d")
        else : 
            publishing = "2023-01-01"

        try : 
            rating = browser.find_element(By.CLASS_NAME, "gd_rating").text
            rating = re.sub(r'[^0-9]', '', rating)

            if rating != '' :
                rating = rating
            else : 
                rating = 0
       
        except :
            
            rating = 0

        review = browser.find_element(By.CLASS_NAME, "gd_reviewCount").text
        review = re.sub(r'[^0-9]', '', review)

        if len(review) > 0 :
            review = int(review)
        else :
            review = 0
         

        sales = browser.find_element(By.CLASS_NAME, 'gd_sellNum').text.split(" ")[2]
        sales = int(sales.replace(",", ""))

        price = browser.find_element(By.CLASS_NAME, "yes_m").text[:-1]
        price = int(price.replace(",", ""))

        full_text = browser.find_element(By.CLASS_NAME, 'gd_best').text
        parts = full_text.split(" | ")

        if len(parts) == 1 :
            ranking = 0
            ranking_weeks = 0
        else : 

            try : 
                ranking_part = parts[0]
                ranking = ''.join(filter(str.isdigit, ranking_part)) #숫자만 추출
            except : 
                raking = 0

            try:
                ranking_weeks_part = parts[1]
                ranking_weeks = ''.join(filter(str.isdigit, ranking_weeks_part.split()[-1]))
            except:
                ranking_weeks = 0

        sql = """INSERT INTO Books(
            title, author, publisher, publishing, rating, review, sales, price, ranking, ranking_weeks
            )
            VALUES(
            %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
            )
            """
        cur.execute(sql, (title, author, publisher, publishing, rating, review, sales, price, ranking, ranking_weeks))
        conn.commit()

        time.sleep(2)